# Tutorial 1: Landcover Classification using Landsat 8

In [1]:
import sys  
sys.path.insert(0, r'F:\raster4ml')

In [2]:
import os
import glob
from raster4ml.preprocessing import stack_bands
from raster4ml.plotting import Map
from raster4ml.features import VegetationIndices
from raster4ml.extraction import batch_extract_values_by_points

## 1. Stack the Bands

First we need to stack all the bands together and make a multispectral image file. The mutispectral image will contain several channels/bands representing reflectance information from different wavelengths. Since the test dataset is downloaded from a Landsat 8 satellite, there are total 11 bands. However, we will only use the first 7 bands as they can accurately define most of the surface objects in terms of reflectance.

To stack the seperate bands into one image, we need to define the paths of all the bands in chronological order (actually any order you want, but remember the orders for future reference).

In [ ]:
# Filter all the files that ends with .TIF
image_dir = r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1'

# Empty list to hold the first 7 bands' paths
bands_to_stack = []
# Loop through 7 times
for i in range(7):
    bands_to_stack.append(os.path.join(image_dir,
                                       f'LC08_L1TP_137045_20210317_20210328_01_T1_B{i+1}.TIF'))
bands_to_stack

In [ ]:
# Use the stack_bands function from raster4ml to do the stacking
stack_bands(image_paths=bands_to_stack,
            out_file=os.path.join(image_dir, 'Stack.tif'))

Let's visualize the image usign the plotting functionality of raster4ml.

In [ ]:
# Define the map instance
m = Map()

In [ ]:
# Add the raster to the map
m.add_raster(image_path=os.path.join(image_dir, 'Stack.tif'), bands=[4, 3, 2])

In [ ]:
m

## 2. Calculate Vegetation Indices

In next step, we need to calculate the vegetation indices from the stacked image. We can do this using `raster4ml.features.VegetationIndices` object. You can provide a list of vegetation index we need to calculate in the object, but the tool can automatically calcualte all the possible vegetation index rasters.  

To do this, we need to provide the path of the stacked image, the corresponding wavelength values and an output directory to save all the indices as rasters. Since this is a Landsat 8 OLI image, we know the band wavelengths. The wavelengths can be inserted as either the `center_wavelengths` as list or the range of wavelengths per band in a list of list. The wavelengths has to be specified in nanometers (nm). The Landsat 8 OLI wavelengths can be seen [here](https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites).  

*Optionally we can provide the `bit_depth` as a parameter. Since we know Landsat 8 data is a 12-bit data, we can provide this information to normalize the image values from 0 to 1.

In [ ]:
# Define the VegetationIndices object
VI = VegetationIndices(image_path=r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\Stack.tif',
                       wavelengths=[[430, 450], [450, 510], [530, 590], [640, 670], [850, 880], [1570, 1650], [2110, 2290]],
                       bit_depth=12)

In [ ]:
# Run the process while providing the output directory
VI.calculate(out_dir=r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\VIs')

Visualize any one of the index to see if the result is ok or not.

In [ ]:
# Add the raster to the map
m.add_raster(image_path=r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\VIs\NDVI.tif')

In [ ]:
m

## 3. Extract Values based on Sample Points

Locate the sample point shapefile in the `data/shapes` folder. The name of the shapefile is `points.shp`. We need to extract the vegetation index values underneath each point in the shapefile and store those index values for Machine Learning training. The shapefile also contains label information. For simplicity, it only has two distinct classes, i.e., `Vegetation` and `Water`.  

For extraction by points, we can use the `raster4ml.extraction.batch_extract_values_by_points` function. This will enable extraction of multiple raster data at once. The function takes `image_paths` as a list, `shape_path` as a string, and a `unique_id` in the shapefile which uniquely represent each point. The function returns a pandas dataframe.

In [ ]:
# Visualize the shapefile onto the map
m.add_shape(shape_path=r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\shapes\points.shp', layer_control=True)

In [ ]:
m

In [3]:
# Find the paths of all the vegetation indices
vi_paths = glob.glob(r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\VIs\*.tif')
vi_paths

['F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\ARI_1.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\ARI_2.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\ARVI.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\CRI_1.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\CRI_2.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\DVI.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\EVI.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\GARI.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\GCI.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\GDVI.tif',
 'F:\\raster4ml\\data\\landsat\\LC08_L1TP_137045_20210317_20210328_01_T1\\VIs\\GEMI.tif',
 'F:\\ras

In [4]:
# Batch extract values by points
values = batch_extract_values_by_points(image_paths=vi_paths,
                                        shape_path=r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\shapes\points.shp',
                                        unique_id='UID')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:29<00:00,  1.62it/s]

6 columns were removed from the dataframe as they had duplicated values.


In [5]:
values

,ARI_1,ARI_2,ARVI,CRI_1,CRI_2,DVI,EVI,GARI,GCI,GDVI,...,PSRI,RDVI,SAVI,SIPI,SR_1,TCARI,TDVI,TVI,VARI,WDRVI
0,-0.062945,-0.108279,0.234300,-0.065647,-0.128593,-0.199707,0.102306,0.213949,-0.212297,-0.463623,...,-0.327823,-0.104673,-0.072355,5.270171,0.895982,0.158350,-0.129161,10.556641,0.169782,-0.696070
1,-0.081763,-0.139712,0.224136,-0.051108,-0.132871,-0.245361,0.114394,0.191005,-0.265274,-0.616943,...,-0.350700,-0.128203,-0.088411,4.550249,0.874438,0.222949,-0.158764,14.863281,0.226522,-0.702291
2,-0.078562,-0.133685,0.156079,-0.039564,-0.118126,-0.395264,0.175768,0.120093,-0.322182,-0.808838,...,-0.329258,-0.202804,-0.137928,3.162446,0.811503,0.248145,-0.252982,16.542969,0.227230,-0.720725
3,0.000882,0.003013,0.162228,-0.010190,-0.009307,0.729492,-14.067797,0.142689,0.274649,0.735840,...,-0.025636,0.295349,0.165779,0.693106,1.271636,-0.003809,0.283973,-0.253906,-0.002432,-0.594480
4,-0.037863,-0.067416,-0.005319,-0.026779,-0.064642,-0.605957,0.372762,-0.028112,-0.321329,-0.843018,...,-0.182404,-0.296846,-0.194758,1.968574,0.746087,0.142236,-0.369330,9.482422,0.108334,-0.740315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.060378,-0.227981,0.509099,-0.041061,-0.101440,1.830078,-2.642041,0.461469,0.712546,1.571045,...,-0.245922,0.765082,0.441218,0.607924,1.940527,0.155420,0.671681,10.361328,0.150049,-0.440813
96,-0.053707,-0.094210,0.223083,-0.068999,-0.122705,-0.169678,0.092413,0.214349,-0.182408,-0.391357,...,-0.309010,-0.088475,-0.060919,5.771223,0.911802,0.133008,-0.108518,8.867188,0.142925,-0.691532
97,-0.044730,-0.080789,0.011139,-0.021725,-0.066455,-0.544189,0.341630,-0.021856,-0.312326,-0.820312,...,-0.185104,-0.266923,-0.175300,2.073127,0.768464,0.165674,-0.330165,11.044922,0.126003,-0.733564
98,-0.015176,-0.029507,-0.061845,-0.015265,-0.030441,-0.604248,0.607601,-0.080921,-0.266599,-0.706787,...,-0.084108,-0.285070,-0.181531,1.575758,0.762908,0.061523,-0.346838,4.101562,0.042080,-0.735235


## 4. Machine Learning Training

Now that we have our data ready, let's build our machine learning model pipelines. We will explore two machine learning models, i.e., Support Vector Machine (SVM) and Random Forest (RF) classification here. Our target variable can be found in the point shapefile as the `Label` column. The independent variables will be the vegetation index values calculated in the last step.

We will utilize functionalities from `scikit-learn` to train the models. `scikit-learn` has an automatic `pipeline` feature that performs several tasks at once. Machine learning models also require **hyperparameter tuning** to fine tune the model. `scikit-learn` has a fetaure for automatically doing that as well using `GridSearchCV`. We will employ all these steps at once using the pipeline.

Therfore install the `scikit-learn` using either `pip` or `conda` in the same environment and import the following modules.

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import geopandas as gpd
import numpy as np

In [8]:
# Read the shapefile to get the points shapefile
# Note that the rows of this shapefile and the extracted values match
shape = gpd.read_file(r"F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\shapes\points.shp")

In [10]:
shape.head()

,Label,UID,geometry
0,Water,0,POINT (193223.422 2349711.302)
1,Water,1,POINT (162754.153 2379518.334)
2,Water,2,POINT (124137.222 2358381.247)
3,Vegetation,3,POINT (224283.022 2424062.863)
4,Water,4,POINT (136454.441 2394163.347)


First, we need to split the dataset into training and testing set. 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(values, shape['Label'].values, test_size=0.3, random_state=42)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (70, 42)
X_test shape: (30, 42)
y_train shape: (70,)
y_test shape: (30,)


Then, we just need to define the `Pipeline`, `GridSearchCV` and the model to do the training.

In [15]:
## Support Vector Machine

# Define pipeline
pipe_svc = Pipeline(steps=[('scaler', MinMaxScaler()), # Scaling the data from 0 to 1
                           ('model', SVC())])

# Define pipeline parameters
# Note that we are only testing 2 hyperparameters, you can do even more or expand the search
param_svc = {'model__gamma': [2**i for i in np.arange(-10, 7, 1, dtype='float')],
             'model__C': [2**i for i in np.arange(-10, 7, 1, dtype='float')]}

# Define grid
grid_svc = GridSearchCV(estimator=pipe_svc,
                        param_grid=param_svc,
                        cv=5, # 5-fold cross validation
                        n_jobs=4) # Paralelly using 4 CPU cores
grid_svc.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model', SVC())]),
             n_jobs=4,
             param_grid={'model__C': [0.0009765625, 0.001953125, 0.00390625,
                                      0.0078125, 0.015625, 0.03125, 0.0625,
                                      0.125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0,
                                      16.0, 32.0, 64.0],
                         'model__gamma': [0.0009765625, 0.001953125, 0.00390625,
                                          0.0078125, 0.015625, 0.03125, 0.0625,
                                          0.125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0,
                                          16.0, 32.0, 64.0]})

In [18]:
## Random Forest Classifier

# Define pipeline
pipe_rfc = Pipeline(steps=[('scaler', MinMaxScaler()), # Scaling the data from 0 to 1
                           ('model', RandomForestClassifier())])

# Define pipeline parameters
# Note that we are only testing 2 hyperparameters, you can do even more or expand the search
param_rfc = {'model__n_estimators': [2**i for i in range(5)],
             'model__max_features': ['sqrt', 'log2']}

# Define grid
grid_rfc = GridSearchCV(estimator=pipe_rfc,
                        param_grid=param_rfc,
                        cv=5, # 5-fold cross validation
                        n_jobs=4) # Paralelly using 4 CPU cores
grid_rfc.fit(X_train, y_train)

C:\Users\sbhadra\.conda\envs\raster4ml_dev\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model', RandomForestClassifier())]),
             n_jobs=4,
             param_grid={'model__max_features': ['auto', 'sqrt', 'log2'],
                         'model__n_estimators': [1, 2, 4, 8, 16]})

Now that we have trained two models, lets check the accuracy score from both models. We can directly use the `grid` objects. If we directly predict from the `grid` object, then it picks out the model with the best hyperparameters and use that for prediction. You can also go into the `grid` object and examine which model to pick and so on. Please refer [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to learn more.

In [19]:
# Predict the test set
y_pred_svc = grid_svc.predict(X_test)
y_pred_rfc = grid_rfc.predict(X_test)

In [22]:
print(f"Accuracy from SVC: {accuracy_score(y_test, y_pred_svc):.2f}")
print(f"Accuracy from RFC: {accuracy_score(y_test, y_pred_rfc):.2f}")

Accuracy from SVC: 0.97
Accuracy from RFC: 1.00
